In [1]:
from llm_evals.candidates import Candidate
from llm_evals.eval import Eval

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
candidate = Candidate.from_yaml('../evals/templates/candidate_openai.yaml')
candidate.to_dict()

{'uuid': '0E454A44-627C-4B56-915B-8DCFE9DF2A55',
 'metadata': {'name': 'OpenAI GPT-3.5-Turbo (1106)'},
 'model_parameters': {'model_name': 'gpt-3.5-turbo-1106',
  'system_message': 'You are a helpful AI assistant.',
  'temperature': 0.01,
  'max_tokens': 4096,
  'seed': 42},
 'candidate_type': 'OPENAI'}

In [3]:
eval_obj = Eval.from_yaml('../evals/evals/mask_emails.yaml')
eval_obj.to_dict()

{'test_sequence': [{'prompt': 'Create a python function called `mask_emails` that uses regex to mask all emails. For each email in the format of `x@y.z`, the local part (`x`) should be masked, but the domain (`@y.z`) should be retained. Use type hints and docstrings.',
   'ideal_response': 'To create a Python function called `mask_emails` that uses regex to mask all emails in a given text, follow these steps:\n\n- Import the `re` module for regular expression operations.\n- Define the function `mask_emails` with type hints for the input and output.\n- Use a regular expression to find all occurrences of emails in the text.\n- Replace the local part of each email (before the \'@\') with an appropriate mask (like asterisks \'*\').\n- Return the modified text with masked emails.\n\nHere\'s how the function might look:\n\n```python\nimport re\n\ndef mask_emails(text: str) -> str:\n    """\n    Masks the local part of all email addresses found in the given text.\n\n    Args:\n    text (str):

In [4]:
result = eval_obj(candidate)
print(result)

EvalResult:
    # of Prompts Tested=2
    Total Response Time (seconds)= 8.2
    # of Response Characters=1,521
    # of Code Blocks Generated=2
    Characters per Second= 185.2
    # of Checks=6
    # of Successful Checks=4
    % of Successful Checks= 66.7%


In [11]:
result.all_checks_results

[PassFailResult(value=True, success=True, metadata={'check_type': 'CONTAINS', 'check_value': 're.sub(', 'check_metadata': {}}),
 PassFailResult(value=False, success=False, metadata={'check_type': 'REGEX', 'check_pattern': 'def mask_email\\(\\s*[a-zA-Z_][a-zA-Z0-9_]*\\s*(\\s*:\\s*[a-zA-Z_][a-zA-Z0-9_]*\\s*)?\\)\\s*:', 'check_metadata': {}}),
 PassFailResult(value=True, success=True, metadata={'check_type': 'PYTHON_CODE_BLOCKS_PRESENT', 'num_code_blocks': 1, 'min_code_blocks': 1, 'code_blocks': ['import re\n\ndef mask_emails(text: str) -> str:\n    """\n    Mask all emails in the given text.\n    \n    Args:\n    text (str): The input text containing emails to be masked.\n    \n    Returns:\n    str: The text with masked emails.\n    """\n    pattern = r\'\\b(\\w+)(@[\\w.-]+)\\b\'\n    masked_text = re.sub(pattern, r\'[MASKED]\\2\', text)\n    return masked_text', '# Test case 1: Single email in the text\nassert mask_emails("Please contact me at john@example.com") == "Please contact me a

In [10]:
result.all_checks_results[-1].metadata

{'check_type': 'PYTHON_CODE_BLOCKS_RUN',
 'num_code_blocks': 2,
 'num_code_blocks_successful': 2,
 'code_blocks': ['import re\n\ndef mask_emails(text: str) -> str:\n    """\n    Mask all emails in the given text.\n    \n    Args:\n    text (str): The input text containing emails to be masked.\n    \n    Returns:\n    str: The text with masked emails.\n    """\n    pattern = r\'\\b(\\w+)(@[\\w.-]+)\\b\'\n    masked_text = re.sub(pattern, r\'[MASKED]\\2\', text)\n    return masked_text',
  '# Test case 1: Single email in the text\nassert mask_emails("Please contact me at john@example.com") == "Please contact me at [MASKED]@example.com"\n\n# Test case 2: Multiple emails in the text\nassert mask_emails("Email me at alice@example.com or bob@example.org") == "Email me at [MASKED]@example.com or [MASKED]@example.org"\n\n# Test case 3: Email with subdomain\nassert mask_emails("Send an email to info@sub.example.com") == "Send an email to [MASKED]@sub.example.com"\n\n# Test case 4: Email with hy